**Решение задачи предсказания правильности выполнения элементов в выступлениях спортсменов дисциплины фигурного катания**
***План работы:***
- загрузка и осмотр данных
- краткий анализ, постановка задачи и веделение таргета
- обучение и валидация моделей МL
- выводы по проекту

In [1]:
import pandas as pd
import numpy as np
from numpy import absolute, mean, std
import warnings
import re
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import StandardScaler, TargetEncoder, OrdinalEncoder
from catboost import CatBoostRegressor
from sklearn.multioutput import MultiOutputRegressor, RegressorChain, RegressorMixin
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)
warnings.filterwarnings('ignore')
random_state = 42

1.***Загрузка и осмотр имеющихся данных:***

In [3]:
units = pd.read_csv('/Users/maiiayakusheva/Downloads/data_01_predict_progress/units.csv')

In [4]:
def initial_analysis(df):
    print('Пример строки датасета:')
    display(df.sample(1))
    print('Общая информация о датасете:')
    print(df.info())
    print('Количество дубликатов:', df.duplicated().sum())
    print('Количество пропусков:')
    print(df.isna().sum()[df.isna().sum()!=0])

In [5]:
initial_analysis(units)

Пример строки датасета:


,id,color,school_id
2310,4020,green,198.0


Общая информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4596 entries, 0 to 4595
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         4596 non-null   int64  
 1   color      4595 non-null   object 
 2   school_id  4007 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 107.8+ KB
None
Количество дубликатов: 0
Количество пропусков:
color          1
school_id    589
dtype: int64


- открыт датасет с информацией о спортсменах, содержащий 4596 строк и 3 столбца с данными. 2 столбца имеют численный тип, один строковый. Дубликатов в данных нет, пропуски (13%) в столбце с идентификатором школы.

In [6]:
turnirs = pd.read_csv('/Users/maiiayakusheva/Downloads/data_01_predict_progress/tournaments.csv')

In [7]:
initial_analysis(turnirs)

Пример строки датасета:


,id,date_start,date_end,origin_id
132,7103,2092-03-25,2092-03-28,0.0


Общая информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          142 non-null    int64  
 1   date_start  142 non-null    object 
 2   date_end    142 non-null    object 
 3   origin_id   142 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.6+ KB
None
Количество дубликатов: 0
Количество пропусков:
Series([], dtype: int64)


- открыт датасет с информацией о турнирах, содержащий 142 строки и 3 столбца с данными. Два столбца имеют численный и два строковый тип. Дубликатов и пропусков в таблице нет.

In [8]:
scores = \
    pd.read_csv('/Users/maiiayakusheva/Downloads/data_01_predict_progress/total_scores.csv')

In [9]:
initial_analysis(scores)

Пример строки датасета:


,id,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str
20905,461967,2861,7114,28.66,28.51,58.87,30.36,0.0,10,8,Короткая программа,< Недокрученный прыжок,8,158.81,8


Общая информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   21301 non-null  int64  
 1   unit_id              21301 non-null  int64  
 2   tournament_id        21301 non-null  int64  
 3   base_score           21301 non-null  float64
 4   components_score     21301 non-null  float64
 5   total_score          21301 non-null  float64
 6   elements_score       21301 non-null  float64
 7   decreasings_score    21301 non-null  float64
 8   starting_place       21301 non-null  int64  
 9   place                21301 non-null  int64  
 10  segment_name         21284 non-null  object 
 11  info                 20720 non-null  object 
 12  overall_place        21301 non-null  int64  
 13  overall_total_score  21284 non-null  float64
 14  overall_place_str    10814 non-null  object 
dtypes: floa

- открыт датасет с общими оценками за турнир, содержащий 21301 строку и 15 столбцов с данными. 12 столбцов имеют численный тип и 2 строковый. Дубликатов в данных нет, пропуски есть (наиболее многочисленные в столбце с комментариями)

In [10]:
turn_scores = \
    pd.read_csv('/Users/maiiayakusheva/Downloads/data_01_predict_progress/tournament_scores.csv')

In [11]:
initial_analysis(turn_scores)

Пример строки датасета:


,id,total_score_id,title,decrease,base_score,goe,avg_score
69465,79547,14154,2Lz,NaN,2.1,0.29,2.39


Общая информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172158 entries, 0 to 172157
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              172158 non-null  int64  
 1   total_score_id  172158 non-null  int64  
 2   title           172158 non-null  object 
 3   decrease        41185 non-null   object 
 4   base_score      172158 non-null  float64
 5   goe             172158 non-null  float64
 6   avg_score       172158 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 9.2+ MB
None
Количество дубликатов: 0
Количество пропусков:
decrease    130973
dtype: int64


- открыт датасет с оценками поэлементно, содержащий 172158 строк и 7 столбцов с данными. 5 столбцов имеют численный и 2 строковый тип. Дубликатов в данных нет, 76% пропусков в столбце с информацией за что снижена оценка
- далее проведу необходимую предобработку данных, для дальнейшего обьединения датасетов по имеющимся ключам

In [12]:
units = units.rename(columns={'id': 'unit_id'})
turnirs = turnirs.rename(columns={'id': 'turnir_id'})
scores = scores.rename(columns={'id': 'total_score_id'})

In [13]:
df_ = units.merge(scores[['total_score_id', 'unit_id', 'tournament_id']], on='unit_id')\
    .merge(turnirs, left_on='tournament_id', right_on='turnir_id')\
        .merge(turn_scores, on='total_score_id')

In [14]:
df_.shape

(172158, 15)

- теперь сгруппирую данные по спортсменам и времени

In [15]:
df_ = df_.sort_values(by='date_start')
df_ = df_.reset_index(drop=True)

In [16]:
df = df_.groupby(['unit_id', 'date_start'], as_index=False)\
    .agg({'color': 'first', 'school_id': 'first', 'origin_id': 'first',\
        'turnir_id': 'first', 'total_score_id': 'unique', 'id': 'unique',\
        'title': 'unique', 'decrease': 'unique', 'base_score': 'unique',\
        'goe': 'unique', 'avg_score': 'unique', 'date_end': 'first'})

- все имеющиеся строковые данные необходимо разделить на отдельные элементы: как то - спортивные элементы, оценки, ошибки и тому подобное. Для этого создам отдельные фреймы данных, которые позже объединю между собой. В связи с отсутствием времени и разностью типов данных пока не обернула все эти действия в единую функцию, планирую доработать этот момент позже

In [17]:
df.total_score_id = [x.tolist() for x in df.total_score_id]
df.total_score_id = [', '.join(map(str, x)) for x in df.total_score_id]
total_score_id = df['total_score_id'].str.split(',', expand=True).fillna(0)
t1 = (df.total_score_id.name*total_score_id.shape[1]).split(df.total_score_id.name[-1])[:-1]
t2 = []
for x in range(len(t1)):
    t2.append(t1[x]+'_'+str(x))
total_score_id.columns = t2

In [18]:
df.id = [', '.join(map(str, x)) for x in df.id]
id = df.id.str.split(',', expand=True).fillna(0)
t1 = (df.id.name*id.shape[1]).split(df.id.name[-1])[:-1]
t2 = []
for x in range(len(t1)):
    t2.append(t1[x]+'_'+str(x))
id.columns = t2

In [19]:
df.title = [', '.join(map(str, x)) for x in df.title]
title = df.title.str.split(',', expand=True).fillna(0)
t1 = (df.title.name*title.shape[1]).split(df.title.name[-1])[:-1]
t2 = []
for x in range(len(t1)):
    t2.append(t1[x]+'_'+str(x))
title.columns = t2

In [20]:
df.decrease = [', '.join(map(str, x)) for x in df.decrease]
decrease = df.decrease.str.split(',', expand=True).fillna(0)
t1 = (df.decrease.name*decrease.shape[1]).split(df.decrease.name[-2:-1])[:-1]
t2 = []
for x in range(len(t1)):
    t2.append(t1[x]+'_'+str(x))
decrease.columns = t2

In [21]:
df.base_score = [', '.join(map(str, x)) for x in df.base_score]
base_score = df.base_score.str.split(',', expand=True).fillna(0)
t1 = (df.base_score.name*base_score.shape[1]).split(df.base_score.name[-2:-1])[:-1]
t2 = []
for x in range(len(t1)):
    t2.append(t1[x]+'_'+str(x))
base_score.columns = t2

In [22]:
df.goe = [', '.join(map(str, x)) for x in df.goe]
goe = df.goe.str.split(',', expand=True).fillna(0)
t1 = (df.goe.name*goe.shape[1]).split(df.goe.name[-1])[:-1]
t2 = []
for x in range(len(t1)):
    t2.append(t1[x]+'_'+str(x))
goe.columns = t2

In [23]:
df.avg_score = [', '.join(map(str, x)) for x in df.avg_score]
avg_score = df.avg_score.str.split(',', expand=True).fillna(0)
t1 = (df.avg_score.name*avg_score.shape[1]).split(df.avg_score.name[-2:-1])[:-1]
t2 = []
for x in range(len(t1)):
    t2.append(t1[x]+'_'+str(x))
avg_score.columns = t2

In [24]:
df_ = pd.concat([\
    df[['unit_id', 'date_start', 'color', 'school_id', 'turnir_id', 'date_end', 'origin_id']],\
        total_score_id, id, title, decrease, base_score, goe, avg_score],\
            axis=1)

In [25]:
def get_my_targets(col):
    temp = col.str.split('+', expand=True)
    return temp

In [26]:
targets = pd.concat([get_my_targets(df_.titl_0), get_my_targets(df_.titl_1),\
    get_my_targets(df_.titl_2), get_my_targets(df_.titl_3),\
        get_my_targets(df_.titl_4), get_my_targets(df_.titl_5),\
            get_my_targets(df_.titl_6), get_my_targets(df_.titl_7),\
                get_my_targets(df_.titl_8), get_my_targets(df_.titl_9),\
                    get_my_targets(df_.titl_10), get_my_targets(df_.titl_11),\
                        get_my_targets(df_.titl_12), get_my_targets(df_.titl_13),\
                            get_my_targets(df_.titl_14), get_my_targets(df_.titl_15),\
    get_my_targets(df_.titl_16), get_my_targets(df_.titl_17),\
        get_my_targets(df_.titl_18), get_my_targets(df_.titl_19),\
            get_my_targets(df_.titl_20), get_my_targets(df_.titl_21),\
                get_my_targets(df_.titl_22), get_my_targets(df_.titl_23)], axis=1)

In [27]:
targets.columns = range(targets.shape[1])
df_ = df_.drop(title.columns.tolist(), axis=1)

In [28]:
targets_ = pd.Series([ ', '.join(z) for z in [[re.sub("[q, None, a, <<, <, !, COMBO, REP, SEQ, *]",\
      '', x) for x in y] for i, y in targets.astype(str).iterrows()]])\
          .str.split(', ', expand=True).fillna(0).replace('', 0)
t1 = ('elements'*targets_.shape[1]).split('elements'[-2:-1])[:-1]
t2 = []
for x in range(len(t1)):
    t2.append(t1[x]+'_'+str(x))
targets_.columns = t2

- первой идеей, как и в принципе первоначально поставленной задачей было решение задачи мультитаргетной классификации по предсказанию цепочки элементов, выполняемых спортсменами в ходе выступления,
- но потом мне пришла идея предсказания количества тех или иных ошибок, допускаемых спорсменом в ходе выступления, за которые снижается общая оценка, поэтому массивом таргетов я сделаю именно количество ошибок в отметках судей и и при помощи модели буду предсказывать именно их количество
- для этого сначала выделю и обработаю их

In [29]:
df_['q'] = [[re.findall('q', x) for x in y] for i, y in targets.astype(str).iterrows()]
df_['q'] = [''.join(z) for z in [[''.join(x) for x in y] for y in df_.q]]
df_['<'] = [[re.findall('<', x) for x in y] for i, y in targets.astype(str).iterrows()]
df_['<'] = [''.join(z) for z in [[''.join(x) for x in y] for y in df_['<']]]
df_['<<'] = [[re.findall('<<', x) for x in y] for i, y in targets.astype(str).iterrows()]
df_['<<'] = [''.join(z) for z in [[''.join(x) for x in y] for y in df_['<<']]]
df_['e'] = [[re.findall('e', x) for x in y] for i, y in targets.astype(str).iterrows()]
df_['e'] = [''.join(z) for z in [[''.join(x) for x in y] for y in df_['e']]]
df_['!'] = [[re.findall('!', x) for x in y] for i, y in targets.astype(str).iterrows()]
df_['!'] = [''.join(z) for z in [[''.join(x) for x in y] for y in df_['!']]]
df_['COMBO'] = [[re.findall('COMBO', x) for x in y] for i, y in targets.astype(str).iterrows()]
df_['COMBO'] = [''.join(z) for z in [[''.join(x) for x in y] for y in df_['COMBO']]]
df_['REP'] = [[re.findall('REP', x) for x in y] for i, y in targets.astype(str).iterrows()]
df_['REP'] = [''.join(z) for z in [[''.join(x) for x in y] for y in df_['REP']]]
df_['SEQ'] = [[re.findall('SEQ', x) for x in y] for i, y in targets.astype(str).iterrows()]
df_['SEQ'] = [''.join(z) for z in [[''.join(x) for x in y] for y in df_['SEQ']]]

In [30]:
#бонус!
df_['x_bonus'] = [[re.findall('x', x) for x in y] for i, y in targets.astype(str).iterrows()]
df_['x_bonus'] = [''.join(z) for z in [[''.join(x) for x in y] for y in df_['x_bonus']]]

In [31]:
df_ = df_.sort_values(by=['unit_id', 'date_start']).reset_index(drop=True)

In [32]:
df_['q'] = [len(x) for x in df_.q]
df_['<'] = [len(x) for x in df_['<']]
df_['<<'] = [len(x) for x in df_['<<']]
df_['e'] = [len(x) for x in df_['e']]
df_['!'] = [len(x) for x in df_['!']]
df_['COMBO'] = [len(x) for x in df_['COMBO']]
df_['REP'] = [len(x) for x in df_['REP']]
df_['SEQ'] = [len(x) for x in df_['SEQ']]

- из-за нехватки времени пока буду просто предсказывать количество ошибок на признаках содержащих только очищенную цепочку элементов по каждому спорсмену, далее при доработке данного проекта, планирую расширить дивизион признаков, и соотвественно улучшить полученную далее метрику

In [33]:
df = pd.concat([targets_, df_[['q', '<', '<<', 'e', '!', 'COMBO', 'REP', 'SEQ']]], axis=1)

In [34]:
train, test = train_test_split(df, test_size=.2, random_state=random_state)

In [35]:
X_tr = train.drop(['q', '<', '<<', 'e', '!', 'COMBO', 'REP', 'SEQ'], axis=1)
y_tr = np.asarray(train[['q', '<', '<<', 'e', '!', 'COMBO', 'REP', 'SEQ']])
X_test = test.drop(['q', '<', '<<', 'e', '!', 'COMBO', 'REP', 'SEQ'], axis=1)
y_test = np.asarray(test[['q', '<', '<<', 'e', '!', 'COMBO', 'REP', 'SEQ']])
X_train, X_val, y_train, y_val = train_test_split(X_tr, y_tr, test_size=.2,\
      random_state=random_state)

In [36]:
X_train.columns = X_train.columns.astype(str)
X_val.columns = X_val.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

In [37]:
cat = []
num = []
for x in X_train.columns:
    if X_train[x].dtype == 'object':
        cat.append(x)
    if X_train[x].dtype in ['int64', 'float64']:
        num.append(x)

In [38]:
X_train[cat] = X_train[cat].astype(str)
X_val[cat] = X_val[cat].astype(str)
X_test[cat] = X_test[cat].astype(str)

In [39]:
enc = OrdinalEncoder().set_output(transform='pandas')
sclr = StandardScaler().set_output(transform='pandas')

In [40]:
X_train_enc = enc.fit_transform(X_train[cat])
X_val_enc = enc.fit_transform(X_val[cat])
X_train = pd.concat([X_train_enc, X_train[num]], axis=1)
X_val = pd.concat([X_val_enc, X_val[num]], axis=1)

In [41]:
X_train = sclr.fit_transform(X_train)
X_val = sclr.transform(X_val)

- сначала попробую предсказать результаты, используя обычный регрессор, работающий с мульти-лейблом:

In [42]:
multioutput = RandomForestRegressor(random_state=random_state)\
    .fit(X_train, y_train)
multioutput.score(X_val, y_val)

0.2303056857612415

- далее буду использовать кросс-валидацию, а также "обертки" для мульти-таргета:

In [43]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=random_state)

In [44]:
model = RandomForestRegressor(random_state=random_state)

In [45]:
n_scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error',\
     cv=cv, n_jobs=-1)

In [46]:
n_scores = absolute(n_scores)
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: 0.813 (0.021)


In [47]:
model = LinearSVR()
wrapper = MultiOutputRegressor(model)

In [48]:
n_scores = cross_val_score(wrapper, X_train, y_train, scoring='neg_mean_absolute_error',\
     cv=cv, n_jobs=-1, verbose=0)

/opt/homebrew/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/svm/_clas

In [49]:
n_scores = absolute(n_scores)
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: 0.863 (0.029)


***Выводы***:
- по результатам данного этапа работы, можно сделать выводы, что данная задача (а именно - предсказание количества ошибок спортсмена в том или ином выступлении) вполне решаема. Далее планирую расширить набор признаков, возможно сделать это в рамках временного лога, что по моему мнению должно улучшить метрику. Также необходимо обернуть все выполненные действия по функциям.